this looks at the clusters from time perm cluster stats and gets their average length across electrodes, and also bins them in a histogram to see the distribution  
also should get the average cluster length

In [1]:
###
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt


['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

c:\Users\jz421\AppData\Local\anaconda3\envs\ieeg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

group_mat = []

adjusted_sampling_freq = 1024

for sub in subjects:
    task = 'GlobalLocal'
    output_name = "Stimulus_fixationCrossBase_1sec_mirror_0to1Test"
    LAB_root = None
    channels = None
    full_trial_base = False

    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Assuming `mat` is your array and `save_dir` is the directory where you want to save it
    mat_save_path = os.path.join(save_dir, f'{output_name}_mat.npy')

    # load the mat array
    mat = np.load(mat_save_path)

    group_mat.append(mat)

# Convert group_mat to a numpy array for calculation
group_mat_np = np.vstack(group_mat)  # Vertically stack the matrices

# Calculate the sum of each row for histogram
row_sums = group_mat_np.sum(axis=1)

row_sums_sig_electrodes = row_sums[row_sums > 0]

# Convert the sums to milliseconds
ms_per_sample = 1000 / adjusted_sampling_freq
row_sums_ms = row_sums * ms_per_sample
row_sums_sig_electrodes_ms = row_sums_sig_electrodes * ms_per_sample

# Assuming `row_sums_ms` contains your data
data = row_sums_ms

# Use numpy.histogram to get the bin edges
counts, bin_edges = np.histogram(data, bins='auto')

# Calculate bin widths as the difference between consecutive bin edges
bin_widths = np.diff(bin_edges)

# Print or analyze the bin widths
print("Bin widths:", bin_widths)

# Plotting the histogram
plt.hist(row_sums_ms, bins='auto')  # 'auto' lets matplotlib decide the number of bins
plt.title('Histogram of Significant Timepoints per Electrode')
plt.xlabel('Total Duration of Significance (ms)')
plt.ylabel('# of Electrodes in bin')
plt.show()

Bin widths: [37.07320602 37.07320602 37.07320602 37.07320602 37.07320602 37.07320602
 37.07320602 37.07320602 37.07320602 37.07320602 37.07320602 37.07320602
 37.07320602 37.07320602 37.07320602 37.07320602 37.07320602 37.07320602
 37.07320602 37.07320602 37.07320602 37.07320602 37.07320602 37.07320602
 37.07320602 37.07320602 37.07320602]


KeyboardInterrupt: 

In [ ]:
def find_clusters_and_onsets(data):
    onsets = []
    for row in data:
        in_cluster = False
        for idx, val in enumerate(row):
            if val > 0 and not in_cluster:
                onsets.append(idx)  # Save the onset time
                in_cluster = True  # Now we're inside a cluster
            elif val == 0:
                in_cluster = False  # Exiting the cluster
    return onsets

# Convert onsets to milliseconds
onsets = find_clusters_and_onsets(group_mat_np)
onsets_ms = [onset * ms_per_sample for onset in onsets]

# Plot histogram of onset times
plt.hist(onsets_ms, bins='auto')
plt.title('Histogram of Cluster Onset Times')
plt.xlabel('Onset Time (ms)')
plt.ylabel('Frequency')
plt.show()

# Assuming `row_sums_ms` contains your data
data = onsets_ms

# Use numpy.histogram to get the bin edges
counts, bin_edges = np.histogram(data, bins='auto')

# Calculate bin widths as the difference between consecutive bin edges
bin_widths = np.diff(bin_edges)

# Print or analyze the bin widths
print("Bin widths:", bin_widths)

plot only sig electrodes

In [4]:
# Assuming `row_sums_ms` contains your data
data = row_sums_sig_electrodes_ms

# Use numpy.histogram to get the bin edges
counts, bin_edges = np.histogram(data, bins='auto')

# Calculate bin widths as the difference between consecutive bin edges
bin_widths = np.diff(bin_edges)

# Print or analyze the bin widths
print("Bin widths:", bin_widths)
# Plotting the histogram
plt.hist(row_sums_sig_electrodes_ms, bins='auto')  # 'auto' lets matplotlib decide the number of bins
plt.title('Histogram of Significant Timepoints per Electrode')
plt.xlabel('Total Duration of Significance (ms)')
plt.ylabel('# of Sig Electrodes in bin')
plt.show()

Bin widths: [83.54048295 83.54048295 83.54048295 83.54048295 83.54048295 83.54048295
 83.54048295 83.54048295 83.54048295 83.54048295 83.54048295]


KeyboardInterrupt: 

get avg and std of row values, to see the avg and std of total sig time across electrodes

In [5]:
avg_total_sig_time = row_sums_sig_electrodes_ms.mean()
std_total_sig_time = row_sums_sig_electrodes_ms.std()

print('avg total sig time: ', avg_total_sig_time)
print('stdev total sig time: ', std_total_sig_time)

avg total sig time:  423.0825892857143
stdev total sig time:  268.4597336795637


now get the avg length and std of each cluster, to get the avg and std of cluster size across electrodes

In [6]:
def find_clusters(row):
    """Find contiguous series of 1's and return their lengths."""
    return [len(cluster) for cluster in ''.join(map(str, row)).split('0') if cluster]

def compute_stats(group_mat_np):
    """Compute average and standard deviation of cluster lengths."""
    cluster_lengths = [find_clusters(row) for row in group_mat_np]
    avg_lengths = [np.mean(clusters) if clusters else 0 for clusters in cluster_lengths]
    std_lengths = [np.std(clusters) if clusters else 0 for clusters in cluster_lengths]
    
    # For significant electrodes only (excluding rows with all zeroes)
    sig_rows = group_mat_np[np.any(group_mat_np == 1, axis=1)]
    sig_cluster_lengths = [find_clusters(row) for row in sig_rows]
    sig_avg_lengths = [np.mean(clusters) if clusters else 0 for clusters in sig_cluster_lengths]
    sig_std_lengths = [np.std(clusters) if clusters else 0 for clusters in sig_cluster_lengths]
    
    return cluster_lengths, avg_lengths, std_lengths, sig_avg_lengths, sig_std_lengths

# Compute the statistics
cluster_lengths, avg_lengths, std_lengths, sig_avg_lengths, sig_std_lengths = compute_stats(group_mat_np) # honestly i only need sig_avg_lengths

# Convert sig_avg_lengths to a numpy array to perform element-wise multiplication
sig_avg_lengths_np = np.array(sig_avg_lengths)
sig_avg_lengths_ms = sig_avg_lengths_np * ms_per_sample

# Create histograms of average cluster lengths
# For all electrodes
# plt.hist(avg_lengths, bins=np.arange(min(avg_lengths), max(avg_lengths) + 1, 1), alpha=0.5, label='All Electrodes')
# For significant electrodes
plt.hist(sig_avg_lengths_ms, bins=np.arange(min(sig_avg_lengths_ms), max(sig_avg_lengths_ms) + 1, 1), alpha=0.5)
plt.title('Histogram of Average Cluster Lengths (ms)')
plt.xlabel('Average Cluster Length (ms)')
plt.ylabel('Number of Electrodes with this Avg Cluster Length')
plt.legend()
plt.show()

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


KeyboardInterrupt: 

In [8]:
min(sig_avg_lengths_ms)

82.03125

In [11]:
non_empty_clusters = [cluster for cluster in cluster_lengths if cluster]

In [14]:
non_empty_clusters_sorted = sorted(non_empty_clusters, key=lambda x: min(x))

In [19]:
non_empty_clusters_sorted_ms = [[length * ms_per_sample for length in cluster] for cluster in non_empty_clusters_sorted]
non_empty_clusters_sorted_ms

[[82.03125],
 [82.03125, 115.234375],
 [88.8671875, 109.375, 83.0078125],
 [102.5390625, 99.609375, 95.703125, 83.984375],
 [83.984375],
 [84.9609375, 114.2578125],
 [84.9609375, 102.5390625],
 [116.2109375, 130.859375, 84.9609375, 86.9140625, 168.9453125],
 [85.9375],
 [85.9375, 101.5625, 90.8203125],
 [87.890625],
 [87.890625],
 [87.890625],
 [87.890625],
 [154.296875, 87.890625, 114.2578125, 89.84375],
 [88.8671875, 108.3984375],
 [88.8671875, 103.515625],
 [88.8671875, 175.78125],
 [125.9765625, 187.5, 88.8671875],
 [89.84375],
 [89.84375],
 [89.84375, 111.328125],
 [90.8203125],
 [90.8203125],
 [90.8203125, 327.1484375],
 [140.625, 90.8203125, 115.234375],
 [183.59375, 90.8203125],
 [530.2734375, 90.8203125],
 [90.8203125],
 [99.609375, 91.796875, 169.921875],
 [92.7734375, 91.796875],
 [91.796875],
 [92.7734375, 284.1796875],
 [92.7734375],
 [92.7734375],
 [301.7578125, 92.7734375],
 [92.7734375, 112.3046875, 102.5390625],
 [121.09375, 123.046875, 93.75],
 [93.75, 274.4140625, 43